In [1]:
import subprocess
import re
from typing import Sequence

from gzip import open as gzip_open

from wordtrie.trie import Trie

In [2]:
with gzip_open("../words.txt.gz", 'rt', encoding="utf-8") as file:
    words = file.read().splitlines()

trie = Trie.from_words(words)

# Wildcard filter

In [3]:
def filter_py_re(words: Sequence[str], pattern: str) -> list[str]:
    return [word for word in words if re.fullmatch(pattern, word)]

In [4]:
def filter_py_re_compiled(words: Sequence[str], pattern: re.Pattern) -> list[str]:
    return [word for word in words if pattern.fullmatch(word)]

In [5]:
def filter_grep(filename: str, pattern: str) -> list[str]:
    result = subprocess.run(
        ['grep', f'^{pattern}$', filename],
        capture_output=True, text=True,
    )
    return result.stdout.splitlines()

In [6]:
def filter_trie(trie: Trie, pattern: str) -> list[str]:
    return list(trie.traverse(pattern))

In [7]:
%timeit filter_py_re(words, r'..SS..')

15.3 ms ± 64.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
pattern_compiled = re.compile(r'..SS..')
%timeit filter_py_re_compiled(words, pattern_compiled)

3.37 ms ± 28.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit filter_grep("../words.txt", r'..SS..')

17.3 ms ± 202 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%timeit -n 10000 filter_trie(trie, r'..SS..')

104 μs ± 7.11 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
filter_trie_result = filter_trie(trie, r'..SS..')

print(filter_trie_result == filter_py_re(words, r'..SS..'))
print(filter_trie_result == filter_py_re_compiled(words, re.compile(r'..SS..')))
print(filter_trie_result == filter_grep('../words.txt', r'..SS..'))

True
True
True


In [12]:
from pprint import pprint


pprint(filter_trie_result, width=100, compact=True)

['BASSED', 'BASSER', 'BASSES', 'BASSET', 'BASSLY', 'BASSOS', 'BISSON', 'BOSSED', 'BOSSER', 'BOSSES',
 'BOSSET', 'BUSSED', 'BUSSES', 'BUSSUS', 'BYSSAL', 'BYSSUS', 'CASSIA', 'CASSIE', 'CASSIS', 'CESSED',
 'CESSER', 'CESSES', 'CISSUS', 'COSSES', 'COSSET', 'COSSIE', 'CUSSED', 'CUSSER', 'CUSSES', 'CUSSOS',
 'DASSIE', 'DESSES', 'DISSED', 'DISSES', 'DOSSAL', 'DOSSED', 'DOSSEL', 'DOSSER', 'DOSSES', 'DOSSIL',
 'EASSEL', 'EASSIL', 'FESSED', 'FESSES', 'FISSLE', 'FOSSAE', 'FOSSAS', 'FOSSED', 'FOSSES', 'FOSSIL',
 'FOSSOR', 'FUSSED', 'FUSSER', 'FUSSES', 'GASSED', 'GASSER', 'GASSES', 'GESSED', 'GESSES', 'GOSSAN',
 'GOSSED', 'GOSSES', 'GOSSIB', 'GOSSIP', 'GUSSET', 'GUSSIE', 'HASSAR', 'HASSEL', 'HASSES', 'HASSLE',
 'HISSED', 'HISSER', 'HISSES', 'HOSSES', 'HUSSAR', 'HUSSES', 'HUSSIF', 'HYSSOP', 'JASSES', 'JASSID',
 'JESSED', 'JESSES', 'JESSIE', 'JISSOM', 'JOSSER', 'JOSSES', 'KISSED', 'KISSEL', 'KISSER', 'KISSES',
 'KOSSES', 'KUSSOS', 'LASSES', 'LASSIE', 'LASSIS', 'LASSOS', 'LASSUS', 'LESSEE', 'LESSEN', 

# Exact match

Both Python's `bisect` and the `look` command assume a sorted word list.

In [13]:
from bisect import bisect_left

def match_py(words: Sequence[str], word: str) -> bool:
    """Determine if `word` is in the sorted sequence `words`."""
    # `bisect_left(words, word)`` returns the maximum i such that words[:i]
    # are all lexicographically less than `word`.  Therefore, if `word` is
    # in `words`, it must be at index `i`.
    i = bisect_left(words, word)
    return i < len(words) and words[i] == word

In [14]:
%timeit match_py(words, "WISHED")

86 ns ± 0.239 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [15]:
%timeit trie.__contains__("WISHED")

0.606 ns ± 0.000399 ns per loop (mean ± std. dev. of 7 runs, 1,000,000,000 loops each)


In [16]:
def match_look(filename: str, word: str) -> bool:
    """Determine if `word` is in the sorted word list `filename` using the `look` command.

    Slower than a pure-Python implementation due to the overhead of spawning a subprocess
    and reading the input file.
    """
    result = subprocess.run(
        ['look', word, filename],
        stdout=subprocess.DEVNULL,
    )
    return result.returncode == 0

In [17]:
%timeit match_look("../words.txt", "WISHED")

29.7 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
match_py(words, "WISHED")

True

# Trie-based functions

In [19]:
"WISHED" in trie

True

In [20]:
print(trie["WO"])

Trie(is_word=True, n_descendants=948)


In [21]:
for n in range(1, 10):
    print(f"WORD + {n} letters:")
    pprint(list(trie.traverse("WORD" + "." * n)), width=100, compact=True)


WORD + 1 letters:
['WORDS', 'WORDY']
WORD + 2 letters:
['WORDED', 'WORDIE']
WORD + 3 letters:
['WORDAGE', 'WORDIER', 'WORDIES', 'WORDILY', 'WORDING', 'WORDISH']
WORD + 4 letters:
['WORDAGES', 'WORDBOOK', 'WORDGAME', 'WORDIEST', 'WORDINGS', 'WORDLESS', 'WORDLORE', 'WORDPLAY',
 'WORDWRAP']
WORD + 5 letters:
['WORDBOOKS', 'WORDBOUND', 'WORDBREAK', 'WORDCOUNT', 'WORDGAMES', 'WORDINESS', 'WORDLORES',
 'WORDPLAYS', 'WORDSMITH', 'WORDWRAPS']
WORD + 6 letters:
['WORDBREAKS', 'WORDCOUNTS', 'WORDLESSLY', 'WORDMONGER', 'WORDSEARCH', 'WORDSMITHS']
WORD + 7 letters:
['WORDINESSES', 'WORDISHNESS', 'WORDMONGERS']
WORD + 8 letters:
['WORDLESSNESS', 'WORDSEARCHES', 'WORDSMITHERY']
WORD + 9 letters:
['WORDISHNESSES']


In [22]:
for n in range(1, 10):
    print(f"{n} letters + SIZE:")
    pprint(list(trie.traverse("." * n + "SIZE")), width=100, compact=True)

1 letters + SIZE:
[]
2 letters + SIZE:
['ASSIZE', 'RESIZE', 'UPSIZE']
3 letters + SIZE:
['CAPSIZE', 'MIDSIZE', 'OUTSIZE', 'UNISIZE']
4 letters + SIZE:
['BITESIZE', 'DOWNSIZE', 'DUMBSIZE', 'GOLDSIZE', 'OVERSIZE', 'PINTSIZE']
5 letters + SIZE:
['ECSTASIZE', 'EMPHASIZE', 'FANTASIZE', 'GALLISIZE', 'MULTISIZE', 'RIGHTSIZE', 'SUPERSIZE',
 'SYNOPSIZE', 'UNDERSIZE']
6 letters + SIZE:
['SYNTHESIZE']
7 letters + SIZE:
['APOTHEOSIZE', 'HYPOSTASIZE', 'HYPOTHESIZE', 'METASTASIZE', 'METATHESIZE', 'REEMPHASIZE']
8 letters + SIZE:
['MISEMPHASIZE', 'PARENTHESIZE', 'RESYNTHESIZE']
9 letters + SIZE:
['OVEREMPHASIZE']


In [23]:
trie["WOR"].__dict__

{'is_word': False,
 'children': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'n_descendants': 323}

# Checking a single word

In [24]:
print(bool(re.fullmatch(r'.ESS..E', "MASSAGE")))
print(bool(re.fullmatch(r'.ESS..E', "MESSAGE")))

False
True


# Attempt to add words

In [25]:
trie.insert("EXISTS")  # Should print an error message

Word 'EXISTS' already in trie.


False

In [26]:
trie.insert("YINCHI")

True

In [27]:
assert "YINCHI" in trie

In [28]:
trie.insert("lowercase")  # Should print an error message

Word 'lowercase' must be capitalized A-Z only.


False